In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from difflib import get_close_matches
from IPython.display import display
from matplotlib.ticker import ScalarFormatter
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import plotly.graph_objects as go
import seaborn as sns
import statistics

## Preprocessing & data cleaning

**steps include:**
- renaming dataset columns to snake_case and english
- checking for missing values
- checking for potential outliers
- adding column country 
- validating text data
- converting date to integer

In [ ]:
df = pd.read_csv("fotbal_prestupy_2000_2019.csv", encoding='utf-8')

In [ ]:
df.head()

In [ ]:
df.rename(columns={"Jméno": "name", 
                   "Pozice": "position", 
                   "Věk": "age", 
                   "Původní tým": "original_team", 
                   "Původní liga": "original_league",
                   "Nový tým": "new_team",
                   "Nová  Liga": "new_league",
                   "Sezóna": "season",
                   "Odhadovaná hodnota": "estimated_value",
                   "Přestupová částka": "transfer_fee"}, inplace=True)
df.head()

## Understanding the dataset

From limited information about the data, some research had to be done to understand and correctly interpret data. Based on empirical evidence, transfer_fee and estimated_value are both in €. For the sake of readability, I will divide those columns by 1000, making it estimated and true value of the player in thousands of euros. 

In [ ]:
df["transfer_fee"] = df["transfer_fee"]/1000
df["estimated_value"] = df["estimated_value"]/1000

### Validating text columns 
First I want to check all the text columns and try to find possible typos which could lead to duplicities. For this I will use difflib function get_close_matches, which will be able to return similar enough cases and should cover typos and mistranslations. 

In [ ]:
text_cols = ["name", "position", "original_team", "original_league", "new_team", "new_league"]

def preprocess_text(column):
    column = column.str.strip() # strip trailing spaces 
    column = column.str.replace("-"," ") # replace - with space
    column = column.str.replace("\n","") # remove newlines
    column = column.str.replace("\t","") # remove tabs
    column = column.str.lower() # all lowercase
    return column

In [ ]:
for col in text_cols:
    df[col] = preprocess_text(df[col])

In [ ]:
def get_similar_entities(unique_names):
    matches= []
    for i, name in enumerate(list(unique_names)):
        possible_match = get_close_matches(name, list(unique_names[i+1:]), cutoff=0.7)
        if possible_match:
            matches.append((name, possible_match))
    return matches

In [ ]:
for col in ["team", "league"]:
    display(f"Similarities for {col}s:")
    display(get_similar_entities(pd.concat([df[f"original_{col}"], df[f"new_{col}"]]).unique()))

### Text cols revision results:
**Revision of close matches found out two mistakes:**
Al Shabab was in several instances named just Shabab
Al Nasr was in several instances named just Nasr

In [ ]:
df.loc[df["original_team"] == "shabab", "original_team"] = "al shabab"
df.loc[df["new_team"] == "shabab", "original_team"] = "al shabab"

df.loc[df["original_team"] == "nasr", "original_team"] = "al nasr"
df.loc[df["new_team"] == "nasr", "new_team"] = "al nasr"

## Validating other columns 
looking into season, age, estimated_value and transfer_fee columns

In [ ]:
df["season"] = df["season"].str.split("-", expand=True)[0] # take the starting year of season
df["season"] = df["season"].astype(int)

### Check validity of ages 

In [ ]:
df["age"].sort_values(ascending=True) # There is one player who is 0 years old

There is one player with age 0. Assuming his parents did not sell his soul to a Saudi Arabian football club, I will impute this value with information available online. If there were more cases than just 1 and imputing with real data was too time consuming or impossible, I would use mean or median imputation. 

From his wikipedia page, I can see that he was 25 years old during his transfer from al shabab to ittihad

In [ ]:
df[df["age"] == 0]

In [ ]:
df.loc[df["age"] == 0, "age"] = 25

In [ ]:
df["age"].isna().sum() 

### Check validity of season
The number and values of seasons matches. No validation to go through here.

In [ ]:
df["season"].unique()

### Check validity of transfer_fee


In [ ]:
df["transfer_fee"].isna().sum() # no missing values

In [ ]:
df.describe().apply(lambda s: s.apply('{0:.2f}'.format))

### Check validity of estimated_value

In [ ]:
df[df.isna().any(axis=1)] #1260 missing values -> all of them are in estimated_value 

There are missing values across all the years except 2015. 
Most of  them in 2000, 2001, 2002, 2003, 2004.

Possible solutions: 
   - Dropping the values
   - mean/median imputation 
   - regression imputation

In [ ]:
df[df.isna().any(axis=1)]["season"].value_counts() 

### Outliers 
Checking for outliers is useful, as their presence could negatively affect models/statistical analysis etc. 
However due to this being only EDA task, while identifying and storing outliers, I will not remove them nor take any measures. On the contrary, I find them quite interesting to look at -> they could give us more insight into why these players are so expensive.
The main idea being not removing outliers from this dataset is that it is NOT randomly sampled - we are sampling the 250 most expensive football transfers in the year, it is natural that we get some extreme values that  all others. 

In [ ]:
def get_outliers_iqr(df, column_name):
    sns.set_palette("turbo")
    fig, ax = plt.subplots(figsize=(10, 6))
    boxplot = ax.boxplot(df[column_name], vert=False, patch_artist=True)

    q1 = df[column_name].quantile(0.25)
    q3 = df[column_name].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr

    lower_bound_ex = q1 - 3 * iqr
    upper_bound_ex = q3 + 3 * iqr

    outliers = df[((df[column_name] < lower_bound) & (df[column_name] >= lower_bound_ex)) | (
            (df[column_name] > upper_bound) & (df[column_name] <= upper_bound_ex))]
    ext_outliers = df[(df[column_name] < lower_bound_ex) | (df[column_name] > upper_bound_ex)]

    ax.scatter(outliers[column_name], [1] * len(outliers),  alpha=0.7, label='Outliers', s=15, marker='o')
    ax.scatter(ext_outliers[column_name], [1] * len(ext_outliers), alpha=0.7, label='Extreme Outliers', s=20, marker='x')

    ax.axvline(lower_bound_ex, color='orange', linestyle='--', label='Lower Bound (3*IQR)')
    ax.axvline(upper_bound_ex, color='orange', linestyle='--', label='Upper Bound (3*IQR)')

    ax.set_yticklabels([column_name])
    ax.set_xlabel('Values')
    ax.set_title(f'Boxplot for {column_name} with Outliers')

    ax.legend()

    ax.xaxis.set_major_formatter(ScalarFormatter(useMathText=False))

    plt.show()
    return outliers, ext_outliers
    

In [ ]:
get_outliers_iqr(df, "age")
outliers, ext_outliers = get_outliers_iqr(df, "transfer_fee")

In [ ]:
ext_outliers.sort_values(by=["transfer_fee"], ascending=False)

### Imputing missing values

For handling missing values I decided to go with regression imputation. Imputing with mean or median values for estimated_value column would make little to no sense as there clearly is a correlation between the *estimated_value* and *transfer_fee* and we should account for that. I decided to go with a simple model with only one explanatory variable and that being *transfer_fee*. Other values such as the player position could be added for more complex model. Needless to say, with R^2 ~ 0.73, the model does a sufficiently good job at imputing the missing values.

In [ ]:
df_copy = pd.get_dummies(df, columns=['position'])
df_complete = df_copy.dropna(subset=['estimated_value'])
df_incomplete = df_copy[df_copy['estimated_value'].isnull()]

X_cols = ['transfer_fee']

X = df_complete[X_cols]
y = df_complete['estimated_value']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

print(f"R-squared (R2): {r2:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
sns.set_palette("turbo")

plt.figure(figsize=(10, 6))

plt.scatter(df_complete['transfer_fee'], y, alpha=0.7, label='Actual Values (Complete Data)')
plt.scatter(df_incomplete['transfer_fee'], model.predict(df_incomplete[X_cols]), color = 'yellow', alpha=0.7, label='Imputed Values')
plt.plot(df_complete['transfer_fee'], model.predict(X), color='red', label='Regression Line')

plt.title('Linear Regression: Estimated Value vs Transfer Fee')
plt.xlabel('Transfer Fee')
plt.ylabel('Estimated Value')
plt.legend()
plt.show()
df.loc[df['estimated_value'].isnull(), 'estimated_value'] = model.predict(df_incomplete[X_cols])

### Deriving feature country - this is me trying to be original
**While league and team may be useful identificators of the background, it may not always be clear where the team is from as we have 615 unique teams. I decided to engineer a features "origin_country" and "target_country", that will represent the countries of the original and target teams, respectively.**

**For this I decided to use GPT 3.5 from OpenAI with the prompt:**
*"You will be presented with a list of football teams. I want you to assign the country of origin to them. Provide me with an output in the form of a python dictionary. E.g. {"Chelsea": "England", "Bayern": "Germany"}. If you are not sure what country the team is from, type "unknown"."*


**Outputs from the language model were saved into *countries.json* file.** 

In [ ]:
pd.concat([df["original_team"], df["new_team"]]).unique()  # output from this list was used in the prompt to generate the feature country

In [ ]:
with open("countries.json", "r", encoding="utf-8") as f:
    countries = json.load(f)

In [ ]:
df_countries = pd.DataFrame.from_dict(countries.items())
df_countries.columns =["team", "country"]
df_countries["team"] = preprocess_text(df_countries["team"])
df_countries["country"] = preprocess_text(df_countries["country"])

In [ ]:
df = pd.merge(df, df_countries, left_on=['new_team'], right_on=['team'], how='left')
df = pd.merge(df, df_countries, left_on=['original_team'], right_on=['team'], how='left', suffixes=('_new', '_original'))
df.drop(['team_new', 'team_original'], axis=1, inplace=True)
df.rename(columns={'country_new': 'new_country', 'country_original': 'original_country'}, inplace=True)

In [ ]:
df.head()

# Analysis of the dataset
## 1. Basic analysis
First we should take a closer look at some basic statistical facts in the dataset. This will help us possibly derive more information as we move forward. We already know some basic info about outlying values and missing values. 

This will include
- distribution of transfer fees
- distribution of age  

In [ ]:
def get_hist(df,col,msg=""):
    sns.set_palette("turbo")
    plt.hist(df[col], bins=20, edgecolor="black")
    plt.title(f"Distribution of values in {col} {msg}:")
    plt.xlabel(col)
    plt.ylabel("Frequency")
    plt.show()
    
    print(f"Mean value of '{col}' {msg}: {np.mean(df[col])}")
    print(f"Median value of '{col}' {msg}: {np.median(df[col])}")
    print(f"Variance value of '{col}' {msg}: {np.var(df[col])}")
    print(f"1st quartile value of '{col}' {msg}: {df[col].quantile(0.25)}")
    print(f"3rd quartile value of '{col}' {msg}: {df[col].quantile(0.75)}")

def get_relative_freq_categorical(df, col):
    position_counts = df[col].value_counts(normalize=True, sort=True)

    sns.set_palette("turbo")

    plt.figure(figsize=(10, 6))  # Adjust the figure size as needed
    bar_plot = sns.barplot(x=position_counts.index, y=position_counts.values)

    plt.xlabel(col)
    plt.ylabel('Relative Frequency')
    plt.title(f'Relative Frequencies of {col}')

    # plt.xticks(rotation=45, ha='right')
    plt.xticks(rotation=90)

    for p in bar_plot.patches:
        bar_plot.annotate(f'{p.get_height()*100:.2f}%', (p.get_x() + p.get_width() / 2., p.get_height()),
                          ha='center', va='center', xytext=(0, 25), textcoords='offset points', rotation = 90)

    # Show the plot
    plt.tight_layout()
    plt.show()

def plot_popular_destinations(dest, top_n):
    country_counts = df.groupby(['season', dest]).size().reset_index(name='counts')
    season_totals = country_counts.groupby('season')['counts'].sum().reset_index(name='total_appearances')

    country_counts = pd.merge(country_counts, season_totals, on='season', how='left')
    country_counts['proportion'] = country_counts['counts'] / country_counts['total_appearances']

    top_countries = country_counts.groupby(dest)['proportion'].sum().nlargest(top_n).index

    top_countries_data = country_counts[country_counts[dest].isin(top_countries)]

    pivot_data = top_countries_data.pivot(index='season', columns=dest, values='proportion').fillna(0)

    plt.figure(figsize=(10, 6))
    for country in pivot_data.columns:
        plt.plot(pivot_data.index, pivot_data[country], label=country, marker='o')

    plt.title(f'Top {top_n} Most Frequent {dest} (Proportional) Over the Years')
    plt.xlabel('Season')
    plt.ylabel('Proportion of Appearances')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

    plt.xticks(pivot_data.index[pivot_data.index % 1 == 0].astype(int))

    plt.grid(True)
    plt.show()

def median_price_per_player(df, col):
    sorted_positions = df.groupby(col)['transfer_fee'].median().sort_values(ascending=False).index
    print(sorted_positions)
    # Create a boxplot using seaborn
    plt.figure(figsize=(10, 6))
    order = sorted_positions if col != 'season' else [i for i in range(2000,2019)]
    sns.boxplot(x=col, y='transfer_fee', data=df, order=order, showfliers = False)
    sns.set_palette("turbo")

    # Rotate x-axis labels
    plt.xticks(rotation=45, ha='right')
    
    plt.title(f'Transfer Fees by ')
    plt.xlabel(col)
    plt.ylabel('Transfer Fee (in thousands of €)')
    
    # Display mean, median, and support values in a separate DataFrame
    summary_df = pd.DataFrame({
        'Position': sorted_positions,
        'Median': df.groupby(col)['transfer_fee'].median().loc[sorted_positions],
        'Mean': df.groupby(col)['transfer_fee'].mean().loc[sorted_positions].round(2),
        'Support': df[col].value_counts().loc[sorted_positions]
    })
    
    # Print the summary DataFrame
    print("Summary DataFrame:")
    print(summary_df)



In [ ]:
get_hist(df,"transfer_fee", "with oultiers")
df_without_outliers = df[~df.isin(pd.concat([outliers, ext_outliers]))].dropna()
get_hist(df_without_outliers,"transfer_fee", "without outliers")
get_hist(outliers,"transfer_fee" ,"only outliers")
get_hist(ext_outliers,"transfer_fee" ,"only extreme outliers")

Distribution of age approx. normally distributed with mean= 24.34 and var = 10.3. 
With IQR being 22 - 27, we can say that 50 % of the most expensive players are between 22 - 27 years old.

In [ ]:
get_hist(df, "age")

More than 25 % of the 250 most expensive football transfers recorded included players playing the 'centre forward' position, then 'centre back' at rougly 15 %, followed by 'central midfield', 'attacking midfield' and 'defensive midfield' at approx 10  %, 9% and 9% respectively. Other positions see a significant dropoff. 'forward', 'defender', 'sweeper' and 'midfielder' see little to no presence in the most exclusive 250. 

In [ ]:
get_relative_freq_categorical(df,"position")

As there is more than plenty of teams and leagues, let's have a look at our added variable country. Analyzing this could give us more comprehensive look into what country's market is the best for high-profile transfers and could be a good intermediate step between analyzing leagues as it could lead us into countries with high amount of expensive transfers.

Looking at the relative frequency plot of new_country in the dataset of transfers, which symbolises the country where the player's new team is based in, we can see that England takes over with 28.70 %, followed by Italy with 16.36 %, Spain at 11.28 %, Germany and France at 9.11 % and 8.55 % respectively. After those leagues, it is only Russia with 5.04 % that clings over 5 %. This can be interpreted as such: In 28.7 % cases from the top 250 most expensive football buyouts from each year ranging from 2000-2018, the team pourchasing the player was based in England. 
This gives us an insight into how strong of a football market England is and how their teams do not fear to go to extensive lengths when purchasing new additions to their teams.


The next plot shows us the evolution on the market over time - at least for the market represented by our limited dataset. I decided to plot the top 6 who all had more than 5% share on the global market. England dominates high-end transfers for all years, the only year it just barely wasn't the top1 market for the most expensive players was 2018, when Italy, with its steady rise from  2014 came in first. In this plot we can also see the decline in high-level purchases by Russian teams, possibly corellating with the sanctions imposed by the EU and USA because of Russia's unlawful 2014 annexation of Ukrainian Crimea. 

In [ ]:
get_relative_freq_categorical(df, "new_country")
plot_popular_destinations("new_country", 6)

plot_popular_destinations("new_country", 2)

While inpecting the relative frequencies of countries in the original_country column, we can see that England tops this chart as well, albeit with slightly less dominance.
Interpreting this plot could be tricky, as one could easily interpret this plot as being the country from which players leave the most. This is not the case. This plot only shows us the home country of most of the teams who agreed to sell their players to another team for a sum that is in top 250 in given year. In this category, Brazil has replaced Russia as the last country above 5 %. This means that while Brazil teams are able to produce top-class expensive players, they are most often than not headed outside Brazil.

In this plot we see a lot more variety in the countries, as talent playing in the 'lesser' countries, when it comes to football culture, tends to go to markets which are able to pay their expensive buyouts. 

In [ ]:
get_relative_freq_categorical(df, "original_country")
plot_popular_destinations("original_country", 6)

Relative frequencies of new_league are looking similarly to new_country analysis from before, but show respective leagues. The chart is once again dominated by the English - the premier league teams account for 26.72 % of the top 250 purchases from the years 2000-2018.

In [ ]:
get_relative_freq_categorical(df, "new_league")
plot_popular_destinations("new_league", 7)

In [ ]:
plot_popular_destinations("new_country", 5)

Let's look into what position dominates the most demanded players in the years 2000-2018. As I am keeping quite a bit of outliers, I decided to stick with median as that will be robust enough for us. 
From this plot we can see that left winger position has the highest median price at 8 200 000 eur with 267 support from the dataset. 
On the other hand, centre forward position has the highest support in our dataset with 1218, but much lower median value of transfer at 6 800 000 eur.
Sweeper, forward, defender and midfielder are represented by units of transactions. As I have no knowledge of football, I will amount this to the unpopular nature of these roles, rather than redundancy with other names (e.g. forward being another name for other position)

In [ ]:
median_price_per_player(df, "position")

Another interesting thing to look at could be the evolution of mean transfer fee over time.
From this we can see that the highest median is in year 2017, lowest in the year 2003 with difference being astonishing. This could be due to inflation, more money being spent in the football industry or generally more competitive environment on the football player market. 

One thing to notice is the increasing variance (volatility) in transfer fees as the time progresses.

In [ ]:
median_price_per_player(df, "season")

### Where are the most expensive players drawn? 

### 1.2 Who are the most expensive players? 

While it may be 

In [ ]:
def plot_popular_destinations(dest, top_n):
    country_counts = df.groupby(['season', dest]).size().reset_index(name='counts')
    season_totals = country_counts.groupby('season')['counts'].sum().reset_index(name='total_appearances')
    
    country_counts = pd.merge(country_counts, season_totals, on='season', how='left')
    country_counts['proportion'] = country_counts['counts'] / country_counts['total_appearances']
    
    top_countries = country_counts.groupby(dest)['proportion'].sum().nlargest(top_n).index
    
    top_countries_data = country_counts[country_counts[dest].isin(top_countries)]
    
    pivot_data = top_countries_data.pivot(index='season', columns=dest, values='proportion').fillna(0)
    
    plt.figure(figsize=(10, 6))
    for country in pivot_data.columns:
        plt.plot(pivot_data.index, pivot_data[country], label=country, marker='o')
    
    plt.title(f'Top {top_n} Most Frequent {dest} (Proportional) Over the Years')
    plt.xlabel('Season')
    plt.ylabel('Proportion of Appearances')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    plt.xticks(pivot_data.index[pivot_data.index % 1 == 0].astype(int))
    
    plt.grid(True)
    plt.show()

### Finding super stars 

In [ ]:
duplicate_names = df[df.duplicated('name', keep=False)]
result = df[df['name'].isin(duplicate_names['name'])]

In [ ]:
result['name'].value_counts()

In [ ]:
result[result["name"] == 'fernando']

In [ ]:
df = df.sort_values(by=['name', 'age'])
df['transfer_fee_diff'] = df.groupby('name')['transfer_fee'].diff()
result = df[df['transfer_fee_diff'] >= 0]
result = result.drop(columns=['transfer_fee_diff'])

In [ ]:
result